<a href="https://colab.research.google.com/github/anweshaban21/finmodel-lab/blob/main/LatticeStructureBullish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LATTICE STRUCTURE FOR BULLISH PERIOD:

---



---






# **Lstm Beta model combined, where the weights of each has been decided by linear regression model** ⏬


In [1]:
pip install yfinance arch statsmodels matplotlib pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 10.8 MB/s eta 0:00:00


In [4]:
def recommend_stocks_lstm_beta(
    stocks, index_symbol, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LinearRegression
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense

    results = []

    # Index returns
    index_data = yf.download(index_symbol, start=start_date, end=end_date)['Close']
    index_returns = np.log(index_data / index_data.shift(1)).dropna()

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 20:
                print(f"⚠️ Not enough data for {stock}")
                continue

            combined = pd.concat([stock_returns, index_returns], axis=1).dropna()
            combined.columns = ['Stock', 'Market']

            # --- Beta estimation ---
            X = sm.add_constant(combined['Market'])
            model = sm.OLS(combined['Stock'], X).fit()
            beta = model.params['Market']

            # --- LSTM volatility estimation ---
            scaler = MinMaxScaler()
            scaled_returns = scaler.fit_transform(stock_returns.values.reshape(-1, 1))

            # Prepare LSTM data
            X_lstm, y_lstm = [], []
            window = 10
            for i in range(window, len(scaled_returns)):
                X_lstm.append(scaled_returns[i - window:i])
                y_lstm.append(scaled_returns[i])
            X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

            # LSTM model
            lstm_model = Sequential()
            lstm_model.add(LSTM(units=50, return_sequences=False, input_shape=(X_lstm.shape[1], 1)))
            lstm_model.add(Dense(1))
            lstm_model.compile(optimizer='adam', loss='mean_squared_error')
            lstm_model.fit(X_lstm, y_lstm, epochs=10, batch_size=8, verbose=0)

            # Predict and compute volatility
            predictions = lstm_model.predict(X_lstm, verbose=0)
            predicted_returns = scaler.inverse_transform(predictions)
            predicted_vol = np.std(predicted_returns)

            avg_return = np.mean(stock_returns)
            std_return = np.std(stock_returns)
            #sharpe_ratio = (avg_return / std_return) if std_return != 0 else 0

            results.append({
                'Stock': stock,
                'Beta': round(beta, 4),
                'LSTM_Volatility': round(predicted_vol, 4),
                'Avg_Return': round(avg_return, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)

    # Drop rows with missing values
    df.dropna(subset=['Beta', 'LSTM_Volatility', 'Avg_Return'], inplace=True)


    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()

    # --- Learn weights via regression ---
    reg = LinearRegression()
    X = df[['Beta', 'LSTM_Volatility']].values
    y = df['Avg_Return'].values

    reg.fit(X, y)
    learned_weights = reg.coef_

    beta_weight, lstm_weight = learned_weights
    print(f"\n📈 Learned Weights from Regression: Beta = {beta_weight:.4f}, LSTM_Vol = {lstm_weight:.4f}")

    # --- Compute risk score using learned weights ---
    df['Risk_Score'] = beta_weight * df['Beta'] + lstm_weight * df['LSTM_Volatility']
    df = df.sort_values(by='Risk_Score')

    print(df[['Stock', 'Beta', 'LSTM_Volatility', 'Avg_Return', 'Risk_Score']].to_string(index=False))


    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f}, Beta: {row['Beta']}, Vol: {row['LSTM_Volatility']}")

    return top


In [5]:
recommend_stocks_lstm_beta(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    index_symbol="^NSEI",
    start_date="2023-01-01",
    end_date="2023-06-30",
    top_n=5
)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3800: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When usin


📈 Learned Weights from Regression: Beta = -0.0019, LSTM_Vol = 0.3199
        Stock   Beta  LSTM_Volatility  Avg_Return    Risk_Score
  RELIANCE.NS 1.3811           0.0017     -0.0002 -2.141046e-03
  HDFCBANK.NS 1.2220           0.0022      0.0003 -1.671782e-03
 ICICIBANK.NS 0.9933           0.0020      0.0003 -1.291163e-03
      INFY.NS 1.3211           0.0041     -0.0013 -1.256567e-03
     WIPRO.NS 0.8984           0.0020     -0.0002 -1.106673e-03
       TCS.NS 0.9475           0.0023     -0.0000 -1.106146e-03
    ZOMATO.NS 1.2712           0.0052      0.0018 -8.076333e-04
     TITAN.NS 0.7233           0.0020      0.0014 -7.662692e-04
        LT.NS 0.7232           0.0026      0.0012 -5.741156e-04
       ITC.NS 0.4793           0.0020      0.0028 -2.919204e-04
ULTRACEMCO.NS 0.5033           0.0023      0.0014 -2.425981e-04
    MARUTI.NS 0.4261           0.0020      0.0011 -1.884968e-04
ASIANPAINT.NS 0.4218           0.0023      0.0008 -8.415783e-05
BAJAJ-AUTO.NS 0.4164           0.0

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3800: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


,Stock,Beta,LSTM_Volatility,Avg_Return,Risk_Score
4,RELIANCE.NS,1.3811,0.0017,-0.0002,-0.002141
2,HDFCBANK.NS,1.2220,0.0022,0.0003,-0.001672
3,ICICIBANK.NS,0.9933,0.0020,0.0003,-0.001291
0,INFY.NS,1.3211,0.0041,-0.0013,-0.001257
13,WIPRO.NS,0.8984,0.0020,-0.0002,-0.001107


# Lstm Beta model combined, where the weights of each has been decided by linear regression model (target variable is sharpe ratio)⏬

In [6]:
def recommend_stocks_lstm_beta(
    stocks, index_symbol, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LinearRegression
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense

    results = []

    # Index returns
    index_data = yf.download(index_symbol, start=start_date, end=end_date)['Close']
    index_returns = np.log(index_data / index_data.shift(1)).dropna()

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 20:
                print(f"⚠️ Not enough data for {stock}")
                continue

            combined = pd.concat([stock_returns, index_returns], axis=1).dropna()
            combined.columns = ['Stock', 'Market']

            # --- Beta estimation ---
            X = sm.add_constant(combined['Market'])
            model = sm.OLS(combined['Stock'], X).fit()
            beta = model.params['Market']

            # --- LSTM volatility estimation ---
            scaler = MinMaxScaler()
            scaled_returns = scaler.fit_transform(stock_returns.values.reshape(-1, 1))

            # Prepare LSTM data
            X_lstm, y_lstm = [], []
            window = 10
            for i in range(window, len(scaled_returns)):
                X_lstm.append(scaled_returns[i - window:i])
                y_lstm.append(scaled_returns[i])
            X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

            # LSTM model
            lstm_model = Sequential()
            lstm_model.add(LSTM(units=50, return_sequences=False, input_shape=(X_lstm.shape[1], 1)))
            lstm_model.add(Dense(1))
            lstm_model.compile(optimizer='adam', loss='mean_squared_error')
            lstm_model.fit(X_lstm, y_lstm, epochs=10, batch_size=8, verbose=0)

            # Predict and compute volatility
            predictions = lstm_model.predict(X_lstm, verbose=0)
            predicted_returns = scaler.inverse_transform(predictions)
            predicted_vol = np.std(predicted_returns)

            avg_return = np.mean(stock_returns)
            sharpe_ratio = avg_return / predicted_vol if predicted_vol != 0 else 0

            results.append({
                'Stock': stock,
                'Beta': round(beta, 4),
                'LSTM_Volatility': round(predicted_vol, 4),
                'Avg_Return': round(avg_return, 4),
                'Sharpe_Ratio': round(sharpe_ratio, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)

    # Drop rows with missing values
    df.dropna(subset=['Beta', 'LSTM_Volatility', 'Sharpe_Ratio'], inplace=True)

    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()

    # --- Learn weights via regression on Sharpe Ratio ---
    reg = LinearRegression()
    X = df[['Beta', 'LSTM_Volatility']].values
    y = df['Sharpe_Ratio'].values
    reg.fit(X, y)
    learned_weights = reg.coef_

    beta_weight, lstm_weight = learned_weights
    print(f"\n📈 Learned Weights from Regression: Beta = {beta_weight:.4f}, LSTM_Vol = {lstm_weight:.4f}")

    # --- Compute risk score using learned weights ---
    df['Risk_Score'] = beta_weight * df['Beta'] + lstm_weight * df['LSTM_Volatility']
    df = df.sort_values(by='Risk_Score')

    print(df[['Stock', 'Beta', 'LSTM_Volatility', 'Avg_Return', 'Sharpe_Ratio', 'Risk_Score']].to_string(index=False))

    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f}, Sharpe Ratio: {row['Sharpe_Ratio']:.4f}, Beta: {row['Beta']}, Vol: {row['LSTM_Volatility']}")

    return top


In [7]:
recommend_stocks_lstm_beta(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    index_symbol="^NSEI",
    start_date="2023-01-01",
    end_date="2023-06-30",
    top_n=5
)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When


📈 Learned Weights from Regression: Beta = -0.8016, LSTM_Vol = 31.3987
        Stock   Beta  LSTM_Volatility  Avg_Return  Sharpe_Ratio  Risk_Score
  RELIANCE.NS 1.3811           0.0016     -0.0002       -0.0939   -1.056806
      INFY.NS 1.3211           0.0042     -0.0013       -0.3008   -0.927075
  HDFCBANK.NS 1.2220           0.0022      0.0003        0.1532   -0.910437
    ZOMATO.NS 1.2712           0.0050      0.0018        0.3579   -0.861958
 ICICIBANK.NS 0.9933           0.0018      0.0003        0.1751   -0.739679
       TCS.NS 0.9475           0.0019     -0.0000       -0.0192   -0.699827
     WIPRO.NS 0.8984           0.0020     -0.0002       -0.1233   -0.657330
     TITAN.NS 0.7233           0.0016      0.0014        0.8704   -0.529535
        LT.NS 0.7232           0.0026      0.0012        0.4735   -0.498057
ULTRACEMCO.NS 0.5033           0.0022      0.0014        0.6278   -0.334351
       ITC.NS 0.4793           0.0019      0.0028        1.4498   -0.324533
    MARUTI.NS 0.4

,Stock,Beta,LSTM_Volatility,Avg_Return,Sharpe_Ratio,Risk_Score
4,RELIANCE.NS,1.3811,0.0016,-0.0002,-0.0939,-1.056806
0,INFY.NS,1.3211,0.0042,-0.0013,-0.3008,-0.927075
2,HDFCBANK.NS,1.2220,0.0022,0.0003,0.1532,-0.910437
14,ZOMATO.NS,1.2712,0.0050,0.0018,0.3579,-0.861958
3,ICICIBANK.NS,0.9933,0.0018,0.0003,0.1751,-0.739679


# BETA GARCH MODEL COMBINED:

In [8]:
def recommend_stocks_beta_garch(
    stocks, index_symbol, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from arch import arch_model
    from sklearn.linear_model import LinearRegression

    results = []

    # Index returns
    index_data = yf.download(index_symbol, start=start_date, end=end_date)['Close']
    index_returns = np.log(index_data / index_data.shift(1)).dropna()

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 30:
                print(f"⚠️ Not enough data for {stock}")
                continue

            combined = pd.concat([stock_returns, index_returns], axis=1).dropna()
            combined.columns = ['Stock', 'Market']

            # --- Beta estimation ---
            X = sm.add_constant(combined['Market'])
            model = sm.OLS(combined['Stock'], X).fit()
            beta = model.params['Market']

            # --- GARCH volatility estimation ---
            garch_model = arch_model(stock_returns * 100, vol='GARCH', p=1, q=1)
            garch_result = garch_model.fit(disp='off')
            garch_volatility = np.mean(np.sqrt(garch_result.conditional_volatility)) / 100  # scale back

            avg_return = np.mean(stock_returns)
            sharpe_ratio = avg_return / garch_volatility if garch_volatility != 0 else 0

            results.append({
                'Stock': stock,
                'Beta': round(beta, 4),
                'GARCH_Volatility': round(garch_volatility, 4),
                'Avg_Return': round(avg_return, 4),
                'Sharpe_Ratio': round(sharpe_ratio, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)

    # Drop rows with missing values
    df.dropna(subset=['Beta', 'GARCH_Volatility', 'Sharpe_Ratio'], inplace=True)

    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()

    # --- Learn weights via regression on Sharpe Ratio ---
    reg = LinearRegression()
    X = df[['Beta', 'GARCH_Volatility']].values
    y = df['Sharpe_Ratio'].values
    reg.fit(X, y)
    learned_weights = reg.coef_

    beta_weight, garch_weight = learned_weights
    print(f"\n📉 Learned Weights from Regression: Beta = {beta_weight:.4f}, GARCH_Vol = {garch_weight:.4f}")

    # --- Compute risk score using learned weights ---
    df['Risk_Score'] = beta_weight * df['Beta'] + garch_weight * df['GARCH_Volatility']
    df = df.sort_values(by='Risk_Score')

    print(df[['Stock', 'Beta', 'GARCH_Volatility', 'Avg_Return', 'Sharpe_Ratio', 'Risk_Score']].to_string(index=False))

    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f}, Sharpe Ratio: {row['Sharpe_Ratio']:.4f}, Beta: {row['Beta']}, Vol: {row['GARCH_Volatility']}")

    return top


In [9]:
recommend_stocks_beta_garch(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    index_symbol="^NSEI",
    start_date="2023-01-01",
    end_date="2023-06-30",
    top_n=5
)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********


📉 Learned Weights from Regression: Beta = -0.2033, GARCH_Vol = 28.4756
        Stock   Beta  GARCH_Volatility  Avg_Return  Sharpe_Ratio  Risk_Score
  RELIANCE.NS 1.3811            0.0104     -0.0002       -0.0145    0.015323
  HDFCBANK.NS 1.2220            0.0105      0.0003        0.0318    0.050520
      INFY.NS 1.3211            0.0117     -0.0013       -0.1077    0.064541
 ICICIBANK.NS 0.9933            0.0095      0.0003        0.0333    0.068547
       TCS.NS 0.9475            0.0102     -0.0000       -0.0036    0.097793
     WIPRO.NS 0.8984            0.0102     -0.0002       -0.0239    0.107776
     TITAN.NS 0.7233            0.0107      0.0014        0.1318    0.157618
        LT.NS 0.7232            0.0109      0.0012        0.1123    0.163333
    ZOMATO.NS 1.2712            0.0156      0.0018        0.1161    0.185742
    MARUTI.NS 0.4261            0.0096      0.0011        0.1097    0.186725
ULTRACEMCO.NS 0.5033            0.0103      0.0014        0.1325    0.190961
    

,Stock,Beta,GARCH_Volatility,Avg_Return,Sharpe_Ratio,Risk_Score
4,RELIANCE.NS,1.3811,0.0104,-0.0002,-0.0145,0.015323
2,HDFCBANK.NS,1.2220,0.0105,0.0003,0.0318,0.050520
0,INFY.NS,1.3211,0.0117,-0.0013,-0.1077,0.064541
3,ICICIBANK.NS,0.9933,0.0095,0.0003,0.0333,0.068547
1,TCS.NS,0.9475,0.0102,-0.0000,-0.0036,0.097793


# BETA LSTM GARCH MODEL COMBINED ⌨

In [10]:
def recommend_stocks_beta_garch_lstm(
    stocks, index_symbol, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from arch import arch_model
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LinearRegression
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense

    results = []

    # Index returns
    index_data = yf.download(index_symbol, start=start_date, end=end_date)['Close']
    index_returns = np.log(index_data / index_data.shift(1)).dropna()

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 30:
                print(f"⚠️ Not enough data for {stock}")
                continue

            combined = pd.concat([stock_returns, index_returns], axis=1).dropna()
            combined.columns = ['Stock', 'Market']

            # --- Beta estimation ---
            X = sm.add_constant(combined['Market'])
            model = sm.OLS(combined['Stock'], X).fit()
            beta = model.params['Market']

            # --- GARCH volatility estimation ---
            garch_model = arch_model(stock_returns * 100, vol='GARCH', p=1, q=1)
            garch_result = garch_model.fit(disp='off')
            garch_vol = np.mean(np.sqrt(garch_result.conditional_volatility)) / 100  # rescale

            # --- LSTM volatility estimation ---
            scaler = MinMaxScaler()
            scaled_returns = scaler.fit_transform(stock_returns.values.reshape(-1, 1))

            X_lstm, y_lstm = [], []
            window = 10
            for i in range(window, len(scaled_returns)):
                X_lstm.append(scaled_returns[i - window:i])
                y_lstm.append(scaled_returns[i])
            X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

            lstm_model = Sequential()
            lstm_model.add(LSTM(units=50, return_sequences=False, input_shape=(X_lstm.shape[1], 1)))
            lstm_model.add(Dense(1))
            lstm_model.compile(optimizer='adam', loss='mean_squared_error')
            lstm_model.fit(X_lstm, y_lstm, epochs=10, batch_size=8, verbose=0)

            predictions = lstm_model.predict(X_lstm, verbose=0)
            predicted_returns = scaler.inverse_transform(predictions)
            lstm_vol = np.std(predicted_returns)

            # --- Sharpe Ratio ---
            avg_return = np.mean(stock_returns)
            avg_vol = np.mean([garch_vol, lstm_vol])
            sharpe_ratio = avg_return / avg_vol if avg_vol != 0 else 0

            results.append({
                'Stock': stock,
                'Beta': round(beta, 4),
                'GARCH_Volatility': round(garch_vol, 4),
                'LSTM_Volatility': round(lstm_vol, 4),
                'Avg_Return': round(avg_return, 4),
                'Sharpe_Ratio': round(sharpe_ratio, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)
    df.dropna(subset=['Beta', 'GARCH_Volatility', 'LSTM_Volatility', 'Sharpe_Ratio'], inplace=True)

    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()

    # --- Learn weights via regression on Sharpe Ratio ---
    reg = LinearRegression()
    X = df[['Beta', 'GARCH_Volatility', 'LSTM_Volatility']].values
    y = df['Sharpe_Ratio'].values
    reg.fit(X, y)
    beta_w, garch_w, lstm_w = reg.coef_

    print(f"\n📊 Learned Weights → Beta = {beta_w:.4f}, GARCH_Vol = {garch_w:.4f}, LSTM_Vol = {lstm_w:.4f}")

    # --- Risk Score ---
    df['Risk_Score'] = (
        beta_w * df['Beta'] +
        garch_w * df['GARCH_Volatility'] +
        lstm_w * df['LSTM_Volatility']
    )

    df = df.sort_values(by='Risk_Score')

    print(df[['Stock', 'Beta', 'GARCH_Volatility', 'LSTM_Volatility', 'Avg_Return', 'Sharpe_Ratio', 'Risk_Score']].to_string(index=False))

    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f}, Sharpe: {row['Sharpe_Ratio']:.4f}, Beta: {row['Beta']}, GARCH Vol: {row['GARCH_Volatility']}, LSTM Vol: {row['LSTM_Volatility']}")

    return top


In [12]:
recommend_stocks_beta_garch_lstm(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    index_symbol="^NSEI",
    start_date="2023-01-01",
    end_date="2023-06-30",
    top_n=5
)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When


📊 Learned Weights → Beta = -0.3355, GARCH_Vol = 118.9242, LSTM_Vol = -146.3686
        Stock   Beta  GARCH_Volatility  LSTM_Volatility  Avg_Return  Sharpe_Ratio  Risk_Score
      INFY.NS 1.3211            0.0117           0.0036     -0.0013       -0.1641    0.421271
  RELIANCE.NS 1.3811            0.0104           0.0018     -0.0002       -0.0248    0.510004
 ICICIBANK.NS 0.9933            0.0095           0.0019      0.0003        0.0554    0.518438
  HDFCBANK.NS 1.2220            0.0105           0.0021      0.0003        0.0529    0.531362
        LT.NS 0.7232            0.0109           0.0031      0.0012        0.1747    0.599905
       TCS.NS 0.9475            0.0102           0.0020     -0.0000       -0.0060    0.602414
     WIPRO.NS 0.8984            0.0102           0.0020     -0.0002       -0.0398    0.618886
    MARUTI.NS 0.4261            0.0096           0.0020      0.0011        0.1814    0.705983
ULTRACEMCO.NS 0.5033            0.0103           0.0022      0.0014       

,Stock,Beta,GARCH_Volatility,LSTM_Volatility,Avg_Return,Sharpe_Ratio,Risk_Score
0,INFY.NS,1.3211,0.0117,0.0036,-0.0013,-0.1641,0.421271
4,RELIANCE.NS,1.3811,0.0104,0.0018,-0.0002,-0.0248,0.510004
3,ICICIBANK.NS,0.9933,0.0095,0.0019,0.0003,0.0554,0.518438
2,HDFCBANK.NS,1.2220,0.0105,0.0021,0.0003,0.0529,0.531362
9,LT.NS,0.7232,0.0109,0.0031,0.0012,0.1747,0.599905


# LSTM GARCH MODEL COMBINED

In [13]:
def recommend_stocks_lstm_garch(
    stocks, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    from arch import arch_model
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LinearRegression
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense

    results = []

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 30:
                print(f"⚠️ Not enough data for {stock}")
                continue

            # --- GARCH volatility estimation ---
            garch_model = arch_model(stock_returns * 100, vol='GARCH', p=1, q=1)
            garch_result = garch_model.fit(disp='off')
            garch_vol = np.mean(np.sqrt(garch_result.conditional_volatility)) / 100

            # --- LSTM volatility estimation ---
            scaler = MinMaxScaler()
            scaled_returns = scaler.fit_transform(stock_returns.values.reshape(-1, 1))

            X_lstm, y_lstm = [], []
            window = 10
            for i in range(window, len(scaled_returns)):
                X_lstm.append(scaled_returns[i - window:i])
                y_lstm.append(scaled_returns[i])
            X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

            lstm_model = Sequential()
            lstm_model.add(LSTM(units=50, return_sequences=False, input_shape=(X_lstm.shape[1], 1)))
            lstm_model.add(Dense(1))
            lstm_model.compile(optimizer='adam', loss='mean_squared_error')
            lstm_model.fit(X_lstm, y_lstm, epochs=10, batch_size=8, verbose=0)

            predictions = lstm_model.predict(X_lstm, verbose=0)
            predicted_returns = scaler.inverse_transform(predictions)
            lstm_vol = np.std(predicted_returns)

            # --- Sharpe Ratio ---
            avg_return = np.mean(stock_returns)
            avg_vol = np.mean([garch_vol, lstm_vol])
            sharpe_ratio = avg_return / avg_vol if avg_vol != 0 else 0

            results.append({
                'Stock': stock,
                'GARCH_Volatility': round(garch_vol, 4),
                'LSTM_Volatility': round(lstm_vol, 4),
                'Avg_Return': round(avg_return, 4),
                'Sharpe_Ratio': round(sharpe_ratio, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)
    df.dropna(subset=['GARCH_Volatility', 'LSTM_Volatility', 'Sharpe_Ratio'], inplace=True)

    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()

    # --- Learn weights via regression on Sharpe Ratio ---
    reg = LinearRegression()
    X = df[['GARCH_Volatility', 'LSTM_Volatility']].values
    y = df['Sharpe_Ratio'].values
    reg.fit(X, y)
    garch_w, lstm_w = reg.coef_

    print(f"\n📊 Learned Weights → GARCH_Vol = {garch_w:.4f}, LSTM_Vol = {lstm_w:.4f}")

    # --- Risk Score ---
    df['Risk_Score'] = (
        garch_w * df['GARCH_Volatility'] +
        lstm_w * df['LSTM_Volatility']
    )

    df = df.sort_values(by='Risk_Score')

    print(df[['Stock', 'GARCH_Volatility', 'LSTM_Volatility', 'Avg_Return', 'Sharpe_Ratio', 'Risk_Score']].to_string(index=False))

    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f}, Sharpe: {row['Sharpe_Ratio']:.4f}, GARCH Vol: {row['GARCH_Volatility']}, LSTM Vol: {row['LSTM_Volatility']}")

    return top


In [15]:
recommend_stocks_lstm_garch(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    start_date="2023-01-01",
    end_date="2023-06-30",
    top_n=5
)

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as th


📊 Learned Weights → GARCH_Vol = 102.3451, LSTM_Vol = -173.8671
        Stock  GARCH_Volatility  LSTM_Volatility  Avg_Return  Sharpe_Ratio  Risk_Score
      INFY.NS            0.0117           0.0041     -0.0013       -0.1594    0.484583
        LT.NS            0.0109           0.0028      0.0012        0.1790    0.628734
 SUNPHARMA.NS            0.0097           0.0020      0.0003        0.0457    0.645014
    MARUTI.NS            0.0096           0.0019      0.0011        0.1835    0.652166
 ICICIBANK.NS            0.0095           0.0018      0.0003        0.0558    0.659318
ASIANPAINT.NS            0.0104           0.0023      0.0008        0.1327    0.664495
     WIPRO.NS            0.0102           0.0021     -0.0002       -0.0396    0.678799
       TCS.NS            0.0102           0.0021     -0.0000       -0.0060    0.678799
ULTRACEMCO.NS            0.0103           0.0021      0.0014        0.2196    0.689034
       ITC.NS            0.0104           0.0021      0.0028      

,Stock,GARCH_Volatility,LSTM_Volatility,Avg_Return,Sharpe_Ratio,Risk_Score
0,INFY.NS,0.0117,0.0041,-0.0013,-0.1594,0.484583
9,LT.NS,0.0109,0.0028,0.0012,0.1790,0.628734
7,SUNPHARMA.NS,0.0097,0.0020,0.0003,0.0457,0.645014
6,MARUTI.NS,0.0096,0.0019,0.0011,0.1835,0.652166
3,ICICIBANK.NS,0.0095,0.0018,0.0003,0.0558,0.659318
